In [ ]:
%pip install mediapipe


In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

## Code from report 1

In [2]:

pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Optical flow parameters
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

cap = cv2.VideoCapture(0)
    
# Initialize variables
old_frame = None
old_gray = None
p0 = None

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    try:
        landmarks = results.pose_landmarks.landmark
        landmark_array = np.array([[lm.x, lm.y] for lm in landmarks])

        if old_frame is None:
            #Initialize points for Lucas-Kanade optical flow
            old_frame = frame.copy()
            old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
            p0 = np.expand_dims(landmark_array, axis=1).astype(np.float32)

        else:
            #Perform Lucas-Kanade optical flow
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

            # Select good points
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            # Update old frame and points
            old_frame = frame.copy()
            old_gray = frame_gray.copy()
            p0 = good_new.reshape(-1, 1, 2)

            # Draw tracks
            for (new, old) in zip(good_new, good_old):
                a, b = new.ravel().astype(int)
                c, d = old.ravel().astype(int)
                cv2.line(frame, (a, b), (c, d), (0, 255, 0), 2)
                cv2.circle(frame, (a, b), 5, (0, 0, 255), -1)

    except:
        pass

    # Render detections
    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                              mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

    cv2.imshow('MediaPipe Feed', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## Only the Pose Landmarks

In [9]:
import cv2
import numpy as np
import mediapipe as mp

# MediaPipe Pose initialization
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Create a VideoCapture object
cap = cv2.VideoCapture(0)

# Get the width and height of the frames
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create a VideoWriter object for the collected video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the video codec (can be changed based on desired format)
out = cv2.VideoWriter('landmarks_video.mp4', fourcc, 30, (frame_width, frame_height), isColor=True)

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Convert the frame to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Pose
    results = pose.process(image)

    try:
        # Create a blank image with a black background
        black_image = np.zeros_like(frame)

        # Render the pose landmarks on the black image
        mp_drawing.draw_landmarks(black_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Write the frame with the pose landmarks to the collected video
        out.write(black_image)

        # Display the frame with the original image and pose landmarks
        cv2.imshow('Pose Landmarks', black_image)

    except:
        pass

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


## What if gumana to para sa LK

In [10]:
import cv2
import numpy as np
import mediapipe as mp

# MediaPipe Pose initialization
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Create a VideoCapture object
cap = cv2.VideoCapture(0)

# Get the width and height of the frames
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create a VideoWriter object for the collected video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the video codec (can be changed based on desired format)
out_landmarks = cv2.VideoWriter('landmarks_video.mp4', fourcc, 30, (frame_width, frame_height), isColor=True)
out_optical_flow = cv2.VideoWriter('optical_flow_video.mp4', fourcc, 30, (frame_width, frame_height), isColor=True)

# Initialize variables for optical flow
prev_gray = None
mask = np.zeros_like(frame)

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Convert the frame to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Pose
    results = pose.process(image)

    try:
        # Create a blank image with a black background
        black_image = np.zeros_like(frame)

        # Render the pose landmarks on the black image
        mp_drawing.draw_landmarks(black_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Write the frame with the pose landmarks to the landmarks video
        out_landmarks.write(black_image)

        # Apply Lucas-Kanade optical flow to track movement
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if prev_gray is None:
            prev_gray = gray

        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # Compute the magnitude and angle of the optical flow
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        # Set the hue according to the angle of the optical flow
        mask[..., 0] = angle * 180 / np.pi / 2

        # Set the value according to the magnitude of the optical flow
        mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

        # Convert HSV to BGR and write to the optical flow video
        flow_bgr = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
        out_optical_flow.write(flow_bgr)

        # Display the frame with the original image and pose landmarks
        cv2.imshow('Pose Landmarks', black_image)

        # Update the previous frame
        prev_gray = gray

    except:
        pass

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
out_landmarks.release()
out_optical_flow.release()
cv2.destroyAllWindows()
